## Load images from record

Herein the data is to be load from TFRecords instead of the system directory.

`inline_augment_images` takes the images in the provided directory and augments them in the directory, removing the old files of the `interim` directory.

`augment_images` also returns a `list` of `dictionaries` which contain information about the images.
This allows for an arbitrary amount of labels saved in the image to be saved without parsing the name of the file or similar.

`encode_record` takes the respective `data_list` and creates TFRecords which are then load later.

This methodology promises to be faster because necessary preprocessing like augmentation and decoding of the image is already done.

Each loaded example of the record is a tensor ready to be put into the training algorithm, with parallel calls and prefetching of data for future steps embedded.

In [1]:
import json
import shutil
import os
from os import path

import cv2
import numpy as np
from tqdm import tqdm

def augment_images_by_label(src_dir, target_dir, label_idx, target_size=(None, None),
    repetitions=1, h_flip=False, v_flip=False, rotation_range=0, quantity=None):
    """Augments the images labelwise

    Arguments:
        src_dir: Where the (raw) images are taken from.
        target_dir: Where the augmented images are going to be safed.
        label: Label to add to the feature_description.
        target_size: Size of the output image.
            If at least one entry is None, the original size is used.
        repetitions: How often should this run over the original dataset?
        h_flip: Is it okay if the images are flipped horizontally?
        v_flip: Is it okay if the images are flipped vertically?
        rotation_range: In what range can the images be rotated (in degrees)?
        quantity: How many images should be taken from the original dataset?
            None means => all.
    """
    data_list = []
    filenames =  list(filter(lambda x: x[-5:] == '.jpeg', os.listdir(src_dir)))
    for filename in filenames[:quantity]:
        image_path = path.join(src_dir, filename)
        image = cv2.imread(image_path)
        if h_flip:
            image = cv2.flip(image, 0)
        if v_flip:
            image = cv2.flip(image, 1)
        if None not in target_size:
            image = cv2.resize(image, target_size)

        angle = np.random.uniform(0.0, rotation_range)
        rotmat = cv2.getRotationMatrix2D(tuple(np.divide(image.shape[:2], 2)), angle, 1.0)
        image = cv2.warpAffine(image, rotmat, image.shape[:2])

        os.makedirs(target_dir, exist_ok=True)
        name = str(len(os.listdir(target_dir)))
        
        # the angle should be labeled between [0, 180)
        angle = int(abs(h_flip*360-abs(v_flip*180-angle))) % 180
        data_list.append({
            'image_path': path.join(target_dir, name + '.jpeg'),
            'label': [label_idx, angle],
        })
        cv2.imwrite(data_list[-1]['image_path'], image)

    return data_list

def augment_images(src_dir, target_dir, repetitions=1, *args, **kwargs):
    """Augments images in src_dir and saves them to target_dir.

    Arguments:
        src_dir: Where the (raw) images are taken from.
        target_dir: Where the augmented images are going to be safed.
        repetitions: How often should this run over the original dataset?
    """

    os.makedirs(target_dir, exist_ok=True)
    data_list = []

    for label_idx, label in enumerate(os.listdir(src_dir)):
        actual_src_dir = path.join(src_dir, label)
        actual_target_dir = path.join(target_dir, label)

        for i in tqdm(range(repetitions)):
            data_list += augment_images_by_label(
                actual_src_dir, actual_target_dir, label_idx, *args, **kwargs)

    np.random.shuffle(data_list)

    with open(path.join(target_dir, 'config.json'), 'w') as config:
        json.dump(data_list, config)

    return data_list

def inline_augment_images(directory, *args, **kwargs):
    """Augments images inplace (source and target directory are the same).

    Arguments:
        directory: source and target directory.

    An intermediate 'directory_tmp' is created.
    It is removed after the operation has finished.
    """

    tmp = directory + '_tmp'
    os.rename(directory, tmp)
    data_list = augment_images(tmp, directory, *args, **kwargs)

    try:
        shutil.rmtree(tmp, ignore_errors=True)
    except OSError as e:
        print ("Error: %s - %s." % (e.filename, e.strerror))

    return data_list

In [2]:
raw = path.join('data', 'raw')
interim = path.join('data', 'interim')
processed = path.join('data', 'processed')

from src.utils import encode_image_data_as_record, reset_and_distribute_data

reset_and_distribute_data(raw, interim, [400,100,0])

train_data_list     = inline_augment_images(path.join(interim, 'train'), target_size=(32, 32))
validate_data_list = inline_augment_images(path.join(interim, 'validate'), target_size=(32, 32))


labels = os.listdir(raw)

train_record    = path.join(processed, 'train.tfrecord')
validate_record = path.join(processed, 'validate.tfrecord')

encode_image_data_as_record(train_data_list, train_record)
encode_image_data_as_record(validate_data_list, validate_record)

100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


In [3]:
import tensorflow as tf

from src.utils import decode_image_record

processed = path.join('data', 'processed')

features = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([2], tf.int64)
}
shape = (32, 32, 1)

def decoder(example):
    feature = tf.io.parse_single_example(example, features)
    image = tf.io.parse_tensor(feature['image'], tf.float32)
    image.set_shape(shape)
    # We only want the 'label_idx'. Not the 'angle'.
    label = feature['label'][0]

    return [image, label]


train_dataset = decode_image_record(path.join(processed, 'train.tfrecord'), decoder, batch_size=32)
validation_dataset = decode_image_record(path.join(processed, 'validate.tfrecord'), decoder, batch_size=10)

In [4]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.optimizers import SGD

model = models.Sequential()
model.add(layers.Flatten(input_shape=(32, 32, 1)))
model.add(layers.Dense(32,'relu'))
model.add(layers.Dense(32,'relu'))
model.add(layers.Dense(3, 'softmax'))

optimizer = SGD(lr=0.005, momentum=0.9, nesterov=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [5]:
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from datetime import datetime
from os import mkdir

log_dir = path.join('logs', 'srp16', datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))

callbacks = [ TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    embeddings_freq=1) ]

history = model.fit(
    train_dataset,
    steps_per_epoch=20,
    epochs=20,
    callbacks=callbacks)

Train for 20 steps
Epoch 1/20
20/20 [==============================] - 1s 33ms/step - loss: 1.0996 - acc: 0.3359
Epoch 2/20
20/20 [==============================] - 0s 6ms/step - loss: 1.0869 - acc: 0.4219
Epoch 3/20
20/20 [==============================] - 0s 6ms/step - loss: 1.0412 - acc: 0.4859
Epoch 4/20
20/20 [==============================] - 0s 6ms/step - loss: 0.9783 - acc: 0.5953
Epoch 5/20
20/20 [==============================] - 0s 6ms/step - loss: 0.9363 - acc: 0.5938
Epoch 6/20
20/20 [==============================] - 0s 5ms/step - loss: 0.8783 - acc: 0.6766
Epoch 7/20
20/20 [==============================] - 0s 6ms/step - loss: 0.8053 - acc: 0.7156
Epoch 8/20
20/20 [==============================] - 0s 6ms/step - loss: 0.7643 - acc: 0.7469
Epoch 9/20
20/20 [==============================] - 0s 5ms/step - loss: 0.6859 - acc: 0.7781
Epoch 10/20
20/20 [==============================] - 0s 6ms/step - loss: 0.6493 - acc: 0.8125
Epoch 11/20
20/20 [==============================